<a href="https://colab.research.google.com/github/npr99/PlanningMethods/blob/master/PLAN604_Comparison_of_two_proportions_tractlevel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of Comparing two Proportions from ACS Data
 
---
This Google Colab Notebook provides a complete workflow (sequence of steps from start to finish) that will allow you to explore two proportions found in ACS Data. 

This notebook compares population proportions found in the 2012 5-year ACS and the 2019 5-year ACS. The research question what census geographies have a stastically significant difference between two proportions found in two different ACS surveys. The 2012 5-year ACS includes random samples of households between 2008, 2009, 2010, 2011, and 2012. The 2019 5-year ACS includes random samples of households between 2015, 2016, 2017, 2018, and 2019.

### Two New python features in this notebook:

#### 1. Resuing code
This notebook introduces Python concepts of reading in and running a python file that includes reusable code. The code in the python file is stored on GitHub and has code that was introduced in earlier notebooks.

When a new pacakge is imported - such as `import pandas as pd` - the notebook is reading in python files that have been shared as open source. The python programming enviroment is simply many python files (`.py`) being referenced. Each (`.py`) is created and maintained by users across the globe.

#### 2. Loops
Loops are a powerful way to repeat functions across different values. In this notebook we will loop over multiple ACS years and multiple variables.


In [1]:
# Python packages required to read in and Census API data
import requests ## Required for the Census API
import pandas as pd # For reading, writing and wrangling data
import matplotlib.pyplot as plt # For plotting and making graphs
import numpy as np

In [6]:
# Downloading and running python script from github
# https://jckantor.github.io/cbe61622/A.02-Downloading_Python_source_files_from_github.html
# Make sure the url is the raw version of the file on GitHub

user = "npr99"
repo = "PlanningMethods"
pyfile = "_planning_methods.py"
url = f"https://raw.githubusercontent.com/{user}/{repo}/master/{pyfile}"
!wget --no-cache --quiet --backups=1 {url}
print("Reading in python file from",url)
#exec(open(pyfile).read())
# Run python file
exec(open(pyfile).read())

SyntaxError: invalid syntax (3031067326.py, line 9)

## Step 1: Obtain Data
The previous step read in a python file with the Obtain Data function presented in the notebook on [Sample Size and Confidence Intervals](https://github.com/npr99/PlanningMethods/blob/master/PLAN604_Population_vs_Sample_USCounties.ipynb) The block of code with the python functions needs to be run first and then the function can be called in future blocks of code.

## Run Obtain Census API for 2012 and 2019 5-year ACS
The next block of code calls the function and gets varaibles related to public transportation use. The code is called in a loop.

For more variables see:

https://www.census.gov/data/developers/data-sets/acs-5year.2012.html

https://api.census.gov/data/2012/acs/acs5/subject/groups/S0801.html


In [3]:
# Loop example
for year in ['2012','2013','2014','2015']:
  print(year)

2012
2013
2014
2015


In [4]:
help(planning_methods)

NameError: name 'planning_methods' is not defined

In [ ]:
# Create an empty "container" to store multiple ACS years for the data
acs_df = {} 

dataset_name = 'acs/acs5/subject'
vintages = ['2012','2019']
get_vars = 'GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M'
state = '48'
census_geography = 'tract:*'
for vintage in vintages:
  print(vintage)
  acs_df[vintage] = planning_methods.obtain_census_api(get_vars = get_vars, 
                                                       state = state,
                                                       census_geography = census_geography,
                                                       dataset_name = dataset_name, 
                                                       vintage = vintage)

2012
Census API data from: https://api.census.gov/data/2012/acs/acs5/subject?get=GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M&in=state:48&in=county:*&for=tract:*
2019
Census API data from: https://api.census.gov/data/2019/acs/acs5/subject?get=GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M&in=state:48&in=county:*&for=tract:*


In [ ]:
acs_df['2012'].head()

,GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M,state,county,tract
0,1400000US48113014132,"Census Tract 141.32, Dallas County, Texas",1154,162,0.8,1.4,48,113,014132
1,1400000US48113014133,"Census Tract 141.33, Dallas County, Texas",2610,340,4.4,3.3,48,113,014133
2,1400000US48113014134,"Census Tract 141.34, Dallas County, Texas",1968,182,1.5,1.8,48,113,014134
3,1400000US48113014135,"Census Tract 141.35, Dallas County, Texas",2201,224,0.6,0.9,48,113,014135
4,1400000US48113014136,"Census Tract 141.36, Dallas County, Texas",3731,316,2.5,1.7,48,113,014136


In [ ]:
acs_df['2019'].head()

,GEO_ID,NAME,S0801_C01_001E,S0801_C01_001M,S0801_C01_009E,S0801_C01_009M,state,county,tract
0,1400000US48061013305,"Census Tract 133.05, Cameron County, Texas",1871,297,0.0,2.2,48,061,013305
1,1400000US48061013309,"Census Tract 133.09, Cameron County, Texas",883,139,0.5,0.9,48,061,013309
2,1400000US48061013402,"Census Tract 134.02, Cameron County, Texas",699,156,6.6,5.9,48,061,013402
3,1400000US48061013500,"Census Tract 135, Cameron County, Texas",722,152,0.0,5.7,48,061,013500
4,1400000US48061012613,"Census Tract 126.13, Cameron County, Texas",2521,293,2.3,3.5,48,061,012613


## Step 2: Clean Data
Data cleaning is an important step in the data science process. This step is often the hardest and most time consuming. 

### 2.1. Clean ACS Data
#### 2.1.1 Make sure variable types are set correctly
For the ACS data we have both population counts and precents. The percentage estiamtes are not integers - they are "floats" which refers the the decimal place that can float in the number between the integer and the fractional part. Here is more information on Python number types https://www.tutorialspoint.com/python/python_numbers.htm 

In [ ]:
Annotation_values = {-999999999 : 'Number of sample cases is too small.',
  -888888888 : 'Estimate is not applicable or not available.',
  -666666666 : 'No sample observations or too few sample observations were available to compute an estimate.',
  -555555555 : 'Estimate is controlled. A statistical test for sampling variability is not appropriate.',
  -333333333 : 'Median falls in the lowest interval or upper interval of an open-ended distribution. A statistical test is not appropriate.',
  -222222222 : 'No sample observations or too few sample observations were available to compute a standard error and thus the margin of error. A statistical test is not appropriate.'}

for Annotation_value in Annotation_values:
  print(Annotation_value)
  print(Annotation_values[Annotation_value])

-999999999
Number of sample cases is too small.
-888888888
Estimate is not applicable or not available.
-666666666
No sample observations or too few sample observations were available to compute an estimate.
-555555555
Estimate is controlled. A statistical test for sampling variability is not appropriate.
-333333333
Median falls in the lowest interval or upper interval of an open-ended distribution. A statistical test is not appropriate.
-222222222
No sample observations or too few sample observations were available to compute a standard error and thus the margin of error. A statistical test is not appropriate.


In [ ]:
import json 

for vintage in vintages:
  for variable in get_vars.split(","):
    variable_metadata_hyperlink = (f'https://api.census.gov/data/{vintage}/{dataset_name}/variables/{variable}.json')
    # Obtain Census API JSON Data
    !wget --no-cache --quiet --backups=1 {variable_metadata_hyperlink}

    with open(f"{variable}.json", "r") as rf:
      variable_metadata = json.load(rf)

    # Find the variable label 
    census_label_string = str(variable_metadata["label"])
    last_exclamation_point_position = census_label_string.rfind("!!")
    if last_exclamation_point_position >= 0:
      last_exclamation_point_position = last_exclamation_point_position + 2
    else:
      last_exclamation_point_position = 0
    label = census_label_string[last_exclamation_point_position:] 

    # Add vintage to label name (skip geo_id and name variables)
    if variable not in ['GEO_ID','NAME']:
      label_addvintage = label + f' {vintage}'
    else:
      label_addvintage = label

    # Add estimate or Margin of Error to label
    last_letter_of_variable = variable[-1]
    if variable not in ['GEO_ID','NAME']:
      if last_letter_of_variable == 'E':
        label_addvintage_addtype = label_addvintage + ' (Estimate)'
      if last_letter_of_variable == 'M':
        label_addvintage_addtype = label_addvintage + ' (MOE)'
    else:
      label_addvintage_addtype = label_addvintage
    print(vintage,"Renameing",variable," = ",label_addvintage_addtype,"Changing type to",variable_metadata["predicateType"])

    # Change variable type
    acs_df[vintage][variable] = acs_df[vintage][variable].astype(variable_metadata["predicateType"])

    # Reset Estimates and MOE with Annotation Values
    Annotation_values = {-999999999 : 'Number of sample cases is too small.',
     -888888888 : 'Estimate is not applicable or not available.',
     -666666666 : 'No sample observations or too few sample observations were available to compute an estimate.',
     -555555555 : 'Estimate is controlled. A statistical test for sampling variability is not appropriate.',
     -333333333 : 'Median falls in the lowest interval or upper interval of an open-ended distribution. A statistical test is not appropriate.',
     -222222222 : 'No sample observations or too few sample observations were available to compute a standard error and thus the margin of error. A statistical test is not appropriate.'}

    for Annotation_value in Annotation_values:
      observations_with_annotation = len(acs_df[vintage].loc[(acs_df[vintage][variable] == Annotation_value)])
      if observations_with_annotation > 0:
        print(observations_with_annotation,"Observations have",Annotation_value)
        print(Annotation_values[Annotation_value])
        print('Replacing values with missing.')
        acs_df[vintage].loc[(acs_df[vintage][variable] == Annotation_value), variable] = np.nan


    acs_df[vintage] = acs_df[vintage].rename(columns={variable: label_addvintage_addtype}) 




2012 Renameing GEO_ID  =  Geography Changing type to string
2012 Renameing NAME  =  Geographic Area Name Changing type to string
2012 Renameing S0801_C01_001E  =  Workers 16 years and over 2012 (Estimate) Changing type to int
2012 Renameing S0801_C01_001M  =  Workers 16 years and over 2012 (MOE) Changing type to int
2012 Renameing S0801_C01_009E  =  Public transportation (excluding taxicab) 2012 (Estimate) Changing type to float
43 Observations have -666666666
No sample observations or too few sample observations were available to compute an estimate.
Replacing values with missing.
2012 Renameing S0801_C01_009M  =  Public transportation (excluding taxicab) 2012 (MOE) Changing type to float
43 Observations have -222222222
No sample observations or too few sample observations were available to compute a standard error and thus the margin of error. A statistical test is not appropriate.
Replacing values with missing.
2019 Renameing GEO_ID  =  Geography Changing type to string
2019 Renamei

#### 2.1.2 Use descriptive statistics to check cleaning
A descriptive statistics table is a great way to check to make sure the variables have been created correctly.

In [ ]:
float_col_list = list(acs_df['2012'].select_dtypes(include=['float']).columns)
table1 = acs_df['2012'][float_col_list].describe().T
varformat = "{:,.2f}" # The variable format adds a comma and rounds up
table_title = "Table 1. Descriptive statistics for variables by county, 2012 5-year ACS."
table1 = table1.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table1

,count,mean,std,min,25%,50%,75%,max
Public transportation (excluding taxicab) 2012 (Estimate),"5,222.00",1.81,3.39,0.00,0.00,0.40,2.10,48.60
Public transportation (excluding taxicab) 2012 (MOE),"5,222.00",2.71,3.55,0.10,1.30,2.00,3.20,100.00


In [ ]:
float_col_list = list(acs_df['2019'].select_dtypes(include=['float']).columns)
table2 = acs_df['2019'][float_col_list].describe().T
varformat = "{:,.2f}" # The variable format adds a comma and rounds up
table_title = "Table 2. Descriptive statistics for variables by county, 2019 5-year ACS."
table2 = table2.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table2

,count,mean,std,min,25%,50%,75%,max
Public transportation (excluding taxicab) 2019 (Estimate),"5,223.00",1.49,2.92,0.00,0.00,0.40,1.80,70.00
Public transportation (excluding taxicab) 2019 (MOE),"5,223.00",2.62,4.02,0.10,1.20,1.90,3.10,100.00


## Step 3: Merge Data together
Merging data together is a powerful tool in data science workflows. Merge data refers to combining data that represent the same unit of analysis. In this example all three datasets represent counties in the US. Each dataframe has a common variable called GEO_ID which uniquely identifies the counties.

Here is a good overview of data merging in python:
https://towardsdatascience.com/why-and-how-to-use-merge-with-pandas-in-python-548600f7e738

### 3.1 - Merge Part 1 - Combine ACS Data

In [ ]:
acs_df['2012'].head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state,county,tract
0,1400000US48113014132,"Census Tract 141.32, Dallas County, Texas",1154,162,0.8,1.4,48,113,014132
1,1400000US48113014133,"Census Tract 141.33, Dallas County, Texas",2610,340,4.4,3.3,48,113,014133
2,1400000US48113014134,"Census Tract 141.34, Dallas County, Texas",1968,182,1.5,1.8,48,113,014134
3,1400000US48113014135,"Census Tract 141.35, Dallas County, Texas",2201,224,0.6,0.9,48,113,014135
4,1400000US48113014136,"Census Tract 141.36, Dallas County, Texas",3731,316,2.5,1.7,48,113,014136


In [ ]:
acs_df['2019'].head()

,Geography,Geographic Area Name,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),state,county,tract
0,1400000US48061013305,"Census Tract 133.05, Cameron County, Texas",1871,297,0.0,2.2,48,061,013305
1,1400000US48061013309,"Census Tract 133.09, Cameron County, Texas",883,139,0.5,0.9,48,061,013309
2,1400000US48061013402,"Census Tract 134.02, Cameron County, Texas",699,156,6.6,5.9,48,061,013402
3,1400000US48061013500,"Census Tract 135, Cameron County, Texas",722,152,0.0,5.7,48,061,013500
4,1400000US48061012613,"Census Tract 126.13, Cameron County, Texas",2521,293,2.3,3.5,48,061,012613


In [ ]:
merge_df = pd.merge(left = acs_df['2012'],
                    right = acs_df['2019'],
                    left_on = ['Geography','Geographic Area Name'],
                    right_on = ['Geography','Geographic Area Name'],
                    how = 'inner')
merge_df.head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,tract_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),state_y,county_y,tract_y
0,1400000US48113014132,"Census Tract 141.32, Dallas County, Texas",1154,162,0.8,1.4,48,113,014132,1293,165,0.9,1.3,48,113,014132
1,1400000US48113014133,"Census Tract 141.33, Dallas County, Texas",2610,340,4.4,3.3,48,113,014133,3442,297,2.9,1.8,48,113,014133
2,1400000US48113014134,"Census Tract 141.34, Dallas County, Texas",1968,182,1.5,1.8,48,113,014134,2239,178,0.5,0.8,48,113,014134
3,1400000US48113014135,"Census Tract 141.35, Dallas County, Texas",2201,224,0.6,0.9,48,113,014135,2506,193,0.0,1.7,48,113,014135
4,1400000US48113014136,"Census Tract 141.36, Dallas County, Texas",3731,316,2.5,1.7,48,113,014136,3908,318,1.7,2.3,48,113,014136


In [ ]:
float_col_list = list(merge_df.select_dtypes(include=['float']).columns)
table3 = merge_df[float_col_list].describe().T
varformat = "{:,.2f}" # The variable format adds a comma and rounds up
table_title = "Table 3. Descriptive statistics for variables by county, 2012 & 2019 5-year ACS."
table3 = table3.style.set_caption(table_title).format(varformat).set_properties(**{'text-align': 'right'})
table3

,count,mean,std,min,25%,50%,75%,max
Public transportation (excluding taxicab) 2012 (Estimate),"5,222.00",1.81,3.39,0.00,0.00,0.40,2.10,48.60
Public transportation (excluding taxicab) 2012 (MOE),"5,222.00",2.71,3.55,0.10,1.30,2.00,3.20,100.00
Public transportation (excluding taxicab) 2019 (Estimate),"5,223.00",1.49,2.92,0.00,0.00,0.40,1.80,70.00
Public transportation (excluding taxicab) 2019 (MOE),"5,223.00",2.62,4.02,0.10,1.20,1.90,3.10,100.00


### 3.3 Clean Up Merge File
Notice that the merge file has several columns (variables) that have repeated information. All three datafiles had a column called `name`. The name column is a good way to see that the merge has worked correctly. In this step we will also rename the columns.

### 3.3.1 Remove repeated columns

In [ ]:
## The state and county columns also match - we can drop the repeated columns
merge_df = merge_df.drop(['state_y','county_y'], axis = 1)
merge_df.head()

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,tract_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),tract_y
0,1400000US48113014132,"Census Tract 141.32, Dallas County, Texas",1154,162,0.8,1.4,48,113,014132,1293,165,0.9,1.3,014132
1,1400000US48113014133,"Census Tract 141.33, Dallas County, Texas",2610,340,4.4,3.3,48,113,014133,3442,297,2.9,1.8,014133
2,1400000US48113014134,"Census Tract 141.34, Dallas County, Texas",1968,182,1.5,1.8,48,113,014134,2239,178,0.5,0.8,014134
3,1400000US48113014135,"Census Tract 141.35, Dallas County, Texas",2201,224,0.6,0.9,48,113,014135,2506,193,0.0,1.7,014135
4,1400000US48113014136,"Census Tract 141.36, Dallas County, Texas",3731,316,2.5,1.7,48,113,014136,3908,318,1.7,2.3,014136


### 3.3.2 Add new variable
Add difference between ACS statiscs

In [ ]:
percent1 = 'Public transportation (excluding taxicab) 2019 (Estimate)'
percent2 = 'Public transportation (excluding taxicab) 2012 (Estimate)'
merge_df.loc[:,'Difference between Percents'] = (merge_df[percent1]) - (merge_df[percent2])
merge_df['Difference between Percents'].describe()

count    5220.000000
mean       -0.311839
std         2.712545
min       -23.000000
25%        -0.800000
50%         0.000000
75%         0.400000
max        51.000000
Name: Difference between Percents, dtype: float64

Add standard error for the pooled estimate

In [ ]:
moe1 = 'Public transportation (excluding taxicab) 2019 (MOE)'
moe2 = 'Public transportation (excluding taxicab) 2012 (MOE)'
census_critical_value = 1.645
merge_df.loc[:,'Standard Error 1'] = (merge_df[moe1])/census_critical_value
merge_df.loc[:,'Standard Error 2'] = (merge_df[moe2])/census_critical_value
merge_df.loc[:,'Difference Standard Error'] = (merge_df['Standard Error 1']**2 + merge_df['Standard Error 2']**2)**(1/2)
merge_df['Difference Standard Error'].describe()

count    5220.000000
mean        0.652009
std         0.488344
min         0.012702
25%         0.354584
50%         0.553273
75%         0.817660
max        11.631907
Name: Difference Standard Error, dtype: float64

Add test statistic for difference

In [ ]:
merge_df.loc[:,'Test Statistic'] = abs(merge_df['Difference between Percents'] / merge_df['Difference Standard Error'])
merge_df['Test Statistic'].describe()

count    5220.000000
mean        5.840989
std        41.656448
min         0.000000
25%         0.000000
50%         0.770617
75%         3.443979
max      2053.062289
Name: Test Statistic, dtype: float64

## Step 4: Explore Data
### 4.1 - Which Census Geographies had significant changes?

In [ ]:
merge_df = merge_df.sort_values(by=['Difference between Percents'], ascending=False)
merge_df.loc[(merge_df['Workers 16 years and over 2012 (Estimate)'] > 4000) &
             (merge_df['Difference between Percents'] > 0) &
             (merge_df['county_x'] == '201') &
             (merge_df['Test Statistic'] >= 3)]

,Geography,Geographic Area Name,Workers 16 years and over 2012 (Estimate),Workers 16 years and over 2012 (MOE),Public transportation (excluding taxicab) 2012 (Estimate),Public transportation (excluding taxicab) 2012 (MOE),state_x,county_x,tract_x,Workers 16 years and over 2019 (Estimate),Workers 16 years and over 2019 (MOE),Public transportation (excluding taxicab) 2019 (Estimate),Public transportation (excluding taxicab) 2019 (MOE),tract_y,Difference between Percents,Standard Error 1,Standard Error 2,Difference Standard Error,Test Statistic
1312,1400000US48201314002,"Census Tract 3140.02, Harris County, Texas",4328,541,16.8,5.1,48,201,314002,4247,571,28.4,7.9,314002,11.6,4.802432,3.100304,0.174941,66.308216
4808,1400000US48201452700,"Census Tract 4527, Harris County, Texas",4968,660,1.5,1.3,48,201,452700,6260,649,6.6,3.6,452700,5.1,2.188450,0.790274,0.429781,11.866512
3840,1400000US48201452100,"Census Tract 4521, Harris County, Texas",4398,626,0.8,0.9,48,201,452100,5718,939,5.8,5.6,452100,5.0,3.404255,0.547112,0.290028,17.239697
4494,1400000US48201533400,"Census Tract 5334, Harris County, Texas",4135,531,0.0,0.9,48,201,533400,4596,648,3.9,2.5,533400,3.9,1.519757,0.547112,0.619104,6.299427
2022,1400000US48201541003,"Census Tract 5410.03, Harris County, Texas",7058,425,0.2,0.4,48,201,541003,7033,818,3.1,2.1,541003,2.9,1.276596,0.243161,0.769499,3.768688
5224,1400000US48201510900,"Census Tract 5109, Harris County, Texas",4145,516,1.0,1.6,48,201,510900,5475,532,3.3,2.9,510900,2.3,1.762918,0.972644,0.496664,4.630898
2212,1400000US48201240902,"Census Tract 2409.02, Harris County, Texas",5359,599,0.5,0.7,48,201,240902,7826,726,2.7,2.7,240902,2.2,1.641337,0.425532,0.589761,3.730324
958,1400000US48201552800,"Census Tract 5528, Harris County, Texas",4208,356,2.4,1.7,48,201,552800,4950,574,4.5,4.2,552800,2.1,2.553191,1.033435,0.363054,5.784261


## Harris County Census Tracts with Increased Public Transportation Ridership

### Census Tract 3140.02, Harris County, Texas

Public Transportation Use increased from 16.8% to 28.4%.	

https://censusreporter.org/profiles/14000US48201314002-census-tract-314002-harris-tx/

[Houston METRO Rail](https://www.google.com/maps/d/viewer?gl=US&ie=UTF8&oe=UTF8&msa=0&mid=1dMDkiFbcozP0fuPtpukqRqVqouQ&ll=29.789473695260714%2C-95.49515840510855&z=13)  had extensive expansion between 2008 and 2019. With new stations added in the area of Census Tract 3140.02.

### Census Tract 5334, Harris County, Texas	

Public Transportation Use increased from 0% to 3.9%.

https://censusreporter.org/profiles/14000US48201533400-census-tract-5334-harris-tx/ 

Houston METRO has [Acres Home Transit Center](https://www.ridemetro.org/pages/TC-AcresHomes.aspx) in Census Tract 5334.  

Acres Homes area of Houston has a long history with providing public transportation to a historically African-American Community. 
https://www.tshaonline.org/handbook/entries/acres-homes-transit-company 

The transit center appears to have minimal ammeneties - a bathroom and some sheltered waiting areas.

https://foursquare.com/v/metro-acres-homes-transit-center/4db479e893a017099dd9ce35

https://en.wikipedia.org/wiki/Acres_Homes,_Houston

### Census Tract 4521, Harris County, Texas	

Public Transportation Use increased from 0.8% to 5.8%.

https://censusreporter.org/profiles/14000US48201452100-census-tract-4521-harris-tx/

Census Tract includes multiple bus routes, is a mile from the [Mission Bend Transit Center](https://www.ridemetro.org/pages/TC-MissionBend.aspx) and appears to have significant empolyement areas.